# Pitcher Data Scraping
The purpose of this notebook is to scrape scouting report data from [Baseball Prospectus' database](https://legacy.baseballprospectus.com/prospects/eyewitness.php), specifically for pitchers.  Initial scraping and scraping of hitter's scouting reports was found in Scouting_Report_Scraping.ipynb.

## Setup/Initiation:

First, importing needed packages:

In [4]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 20,10
import numpy as np
import glob
from scipy import stats
from bs4 import BeautifulSoup
import requests
import re
from IPython.core.display import display, HTML    # make sure Jupyter knows to display it as HTML

To scrape this data, I'll need the initial list of links I scraped:

In [6]:
with open('./Data_Files/initial_full_df_links.pickle', 'rb') as to_read:
    initial_df = pickle.load(to_read)

In [7]:
initial_df.head()

,Player,Primary Pos,Evaluator,Report Date,OFP,Player_Page_Link
0,CJ Abrams,SS,Keanan Lamb,06/01/2019,50,https://legacy.baseballprospectus.com/prospect...
1,Albert Abreu,P,Mauricio Rubio Jr.,08/27/2016,60,https://legacy.baseballprospectus.com/prospect...
2,Osvaldo Abreu,SS,Tucker Blair,05/04/2015,40,https://legacy.baseballprospectus.com/prospect...
3,Albert Abreu,P,Grant Jones,00/00/0000,60,https://legacy.baseballprospectus.com/prospect...
4,Albert Abreu,P,John Eshleman,12/13/2017,60,https://legacy.baseballprospectus.com/prospect...


In [8]:
initial_df.shape

(1074, 6)

Now I can move on to scraping pitcher reports using that list of links.  Pitchers will be ones that have 'eyewitness_pit' in the link (as opposed to 'eyewitness_bat' for hitters).

## Scouting of Pitcher Reports

Using [Jose Leclerc's page as a reference](https://legacy.baseballprospectus.com/prospects/eyewitness_pit.php?reportid=112), these are the fields I'll want in my data:
- Initial Info: Name, Born, Bats, Throws, Height, Weight
- Eval Info: Evaluator, Report Date, Affiliate, Dates Seen, OFP, MLB ETA
- Mechanics
- Pitch Data: For each pitch, Type, Future Grade, Sitting Velocity, Peak Velocity, Report
- Overall Report